# Full Market Scanner

Comprehensive scanner for all market categories with automated report generation.

### Coverage
- **S&P 500**: ~500 stocks
- **NASDAQ 100**: ~100 stocks  
- **Portfolio**: Custom watchlist from `stocks.txt`

### Output
- **Excel**: Single-tab FULL HOLD + ADD signals (S&P 500 & NASDAQ 100), Multi-tab signals (Portfolio)
- **PDF**: Detailed research reports with support/resistance levels
- **Cleanup**: Automatically archives old scans (keeps most recent only)

### Runtime (with rate limit protection)
- **First run today**: ~15-25 minutes (fetches fresh data with delays)
- **Second run today**: ~2-3 minutes (uses 24hr cache)
- S&P 500: ~6-8 minutes | NASDAQ 100: ~1-2 minutes | Portfolio: ~30 seconds

In [1]:
# ============================================================================
# CELL 2: Import all required libraries and scanner functions
# ============================================================================

# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import importlib

# Import scanner functions
import full_scanner
importlib.reload(full_scanner)
from full_scanner import (get_sp500_tickers, get_nasdaq100_tickers, scan_stocks, 
                          filter_buy_signals, create_excel_output, create_pdf_report, 
                          create_portfolio_excel, cleanup_old_scans)

# Display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

---
## Step 1: Scan S&P 500
Scanning ~500 stocks (6-8 minutes first run, instant if cached)

In [2]:
# ============================================================================
# CELL 4: S&P 500 Scanner - Fetches ~503 stocks and scans for signals
# ============================================================================

print("="*80)
print("SCAN 1 of 3: S&P 500")
print("="*80 + "\n")

start_time = datetime.now()
sp500_tickers = get_sp500_tickers()
print(f"Found {len(sp500_tickers)} S&P 500 stocks\n")

sp500_results = scan_stocks(sp500_tickers, category="S&P 500", daily_bars=60, weekly_bars=52, concurrency=2)
sp500_elapsed = (datetime.now() - start_time).total_seconds()

print(f"\n✓ S&P 500 scan completed in {sp500_elapsed:.1f} seconds")

SCAN 1 of 3: S&P 500

✓ Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

🔍 Scanning 503 S&P 500 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
✓ [4/503] ABBV   -> FULL HOLD + ADD      $220.18
✓ [9/503] AFL    -> FULL HOLD + ADD      $112.19
✓ [14/503] ALB    -> FULL HOLD + ADD      $146.13
✓ [18/503] ALL    -> FULL HOLD + ADD      $205.57
✓ [20/503] GOOGL  -> FULL HOLD + ADD      $316.54
✓ [21/503] GOOG   -> FULL HOLD + ADD      $317.32
✓ [23/503] AMZN   -> FULL HOLD + ADD      $233.06
✓ [27/503] AXP    -> FULL HOLD + ADD      $379.80
✓ [28/503] AIG    -> FULL HOLD + ADD      $84.38
✓ [32/503] AME    -> FULL HOLD + ADD      $211.31
✓ [33/503] AMGN   -> FULL HOLD + ADD      $320.72
✓ [34/503] APH    -> FULL HOLD + ADD      $139.88
✓ [35/503] ADI    -> FULL HOLD + ADD      $277.29
✓ [38/503] APO    -> FULL HOLD + ADD      $149.76
✓ [39/503] AAPL   -> FULL HOLD + ADD      $267.26
✓ [40/503] AMAT   -> FULL HOLD + ADD      $284.32
✓ [41/503

---
## Step 2: Scan NASDAQ 100
Scanning ~100 stocks (1-2 minutes first run, instant if cached)

In [3]:
# ============================================================================
# CELL 6: NASDAQ 100 Scanner - Fetches ~101 stocks and scans for signals
# ============================================================================

print("\n" + "="*80)
print("SCAN 2 of 3: NASDAQ 100")
print("="*80 + "\n")

start_time = datetime.now()
nasdaq100_tickers = get_nasdaq100_tickers()
print(f"Found {len(nasdaq100_tickers)} NASDAQ 100 stocks\n")

nasdaq100_results = scan_stocks(nasdaq100_tickers, category="NASDAQ 100", daily_bars=60, weekly_bars=52, concurrency=2)
nasdaq100_elapsed = (datetime.now() - start_time).total_seconds()

print(f"\n✓ NASDAQ 100 scan completed in {nasdaq100_elapsed:.1f} seconds")


SCAN 2 of 3: NASDAQ 100

✓ Loaded 101 NASDAQ 100 tickers

Found 101 NASDAQ 100 stocks

🔍 Scanning 101 NASDAQ 100 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
✓ [4/101] GOOGL  -> FULL HOLD + ADD      $316.54
✓ [5/101] GOOG   -> FULL HOLD + ADD      $317.32
✓ [6/101] AMZN   -> FULL HOLD + ADD      $233.06
✓ [8/101] AMGN   -> FULL HOLD + ADD      $320.72
✓ [9/101] ADI    -> FULL HOLD + ADD      $277.29
✓ [10/101] AAPL   -> FULL HOLD + ADD      $267.26
✓ [11/101] AMAT   -> FULL HOLD + ADD      $284.32
✓ [12/101] APP    -> FULL HOLD + ADD      $632.91
✓ [15/101] ASML   -> FULL HOLD + ADD      $1228.19
✓ [16/101] AZN    -> FULL HOLD + ADD      $91.22
✓ [21/101] BKNG   -> FULL HOLD + ADD      $5367.37
✓ [26/101] CSCO   -> FULL HOLD + ADD      $75.58
✓ [27/101] CCEP   -> FULL HOLD + ADD      $87.56
✓ [28/101] CTSH   -> FULL HOLD + ADD      $81.63
✓ [30/101] CEG    -> FULL HOLD + ADD      $354.94
✓ [35/101] CSX    -> FULL HOLD + ADD      $35.91
✓

---
## Step 3: Scan Portfolio
Scanning custom watchlist from `stocks.txt` (~30 seconds, instant if cached)

In [4]:
# ============================================================================
# CELL 8: Portfolio Scanner - Loads stocks.txt and scans your watchlist
# Note: Skips lines starting with # (comments) or [ (basket headers)
# ============================================================================

print("\n" + "="*80)
print("SCAN 3 of 3: PORTFOLIO STOCKS")
print("="*80 + "\n")

# Load portfolio tickers
stocks_file = Path('stocks.txt')
if stocks_file.exists():
    with open(stocks_file) as f:
        portfolio_tickers = [line.strip() for line in f 
                            if line.strip() 
                            and not line.startswith('#')
                            and not line.startswith('[')]  # Skip basket lines
    print(f"✓ Loaded {len(portfolio_tickers)} portfolio tickers from stocks.txt\n")
else:
    portfolio_tickers = []
    print("⚠️  stocks.txt not found, skipping portfolio scan\n")

if portfolio_tickers:
    start_time = datetime.now()
    portfolio_results = scan_stocks(portfolio_tickers, category="Portfolio", daily_bars=60, weekly_bars=52, concurrency=2)
    portfolio_elapsed = (datetime.now() - start_time).total_seconds()
    print(f"\n✓ Portfolio scan completed in {portfolio_elapsed:.1f} seconds")
else:
    portfolio_results = pd.DataFrame()
    portfolio_elapsed = 0


SCAN 3 of 3: PORTFOLIO STOCKS

✓ Loaded 13 portfolio tickers from stocks.txt

🔍 Scanning 13 Portfolio stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
✓ [1/13] TSLA   -> FULL HOLD + ADD      $451.67
✓ [2/13] NVDA   -> FULL HOLD + ADD      $188.12
✓ [6/13] ASML   -> FULL HOLD + ADD      $1228.19
✓ [8/13] PLTR   -> FULL HOLD + ADD      $174.04
✓ [10/13] MRVL   -> FULL HOLD + ADD      $90.23

✓ Scan complete: 13 analyzed, 5 FULL HOLD + ADD signals found


✓ Portfolio scan completed in 5.0 seconds


---
## Step 4: Generate All Reports
Creating Excel files and PDF research documents for each category

In [5]:
# ============================================================================
# CELL 10: Report Generator - Creates 6 files (3 Excel + 3 PDF)
# - S&P 500: sp500_analysis_*.xlsx + PDF (1 tab: FULL HOLD + ADD only)
# - NASDAQ 100: nasdaq100_analysis_*.xlsx + PDF (1 tab: FULL HOLD + ADD only)
# - Portfolio: portfolio_scanner_*.xlsx + PDF (5 tabs: All signals)
# ============================================================================

print("\n" + "="*80)
print("GENERATING REPORTS")
print("="*80 + "\n")

results_dir = Path.cwd() / 'scanner_results'
results_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# === S&P 500 Reports ===
if not sp500_results.empty:
    sp500_buy = filter_buy_signals(sp500_results, 'FULL HOLD + ADD')
    print(f"🎯 S&P 500: {len(sp500_buy)} FULL HOLD + ADD signals found")
    
    if not sp500_buy.empty:
        xlsx_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_sp500_{timestamp}.pdf'
        
        create_excel_output(sp500_buy, xlsx_path, category="S&P 500")
        create_pdf_report(sp500_buy, sp500_results, pdf_path, timestamp, category="S&P 500")
        print(f"  ✓ Excel: {xlsx_path.name}")
        print(f"  ✓ PDF: {pdf_path.name}\n")

# === NASDAQ 100 Reports ===
if not nasdaq100_results.empty:
    nasdaq100_buy = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD')
    print(f"🎯 NASDAQ 100: {len(nasdaq100_buy)} FULL HOLD + ADD signals found")
    
    if not nasdaq100_buy.empty:
        xlsx_path = results_dir / f'nasdaq100_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_nasdaq100_{timestamp}.pdf'
        
        create_excel_output(nasdaq100_buy, xlsx_path, category="NASDAQ 100")
        create_pdf_report(nasdaq100_buy, nasdaq100_results, pdf_path, timestamp, category="NASDAQ 100")
        print(f"  ✓ Excel: {xlsx_path.name}")
        print(f"  ✓ PDF: {pdf_path.name}\n")

# === Portfolio Reports ===
if not portfolio_results.empty:
    portfolio_buy = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD')
    print(f"🎯 Portfolio: {len(portfolio_results)} stocks scanned (all included)")
    print(f"  - {len(portfolio_buy)} with FULL HOLD + ADD signal")
    
    xlsx_path = results_dir / f'portfolio_scanner_{timestamp}.xlsx'
    pdf_path = results_dir / f'scanner_report_portfolio_{timestamp}.pdf'
    
    create_portfolio_excel(portfolio_results, xlsx_path, category="Portfolio")
    create_pdf_report(portfolio_buy, portfolio_results, pdf_path, timestamp, category="Portfolio")
    print(f"  ✓ Excel: {xlsx_path.name}")
    print(f"  ✓ PDF: {pdf_path.name}\n")

# Cleanup old scans
print("📁 Managing scan history...")
cleanup_old_scans(results_dir, max_files=1)


GENERATING REPORTS

🎯 S&P 500: 202 FULL HOLD + ADD signals found
✓ Excel file created: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260106_1456.xlsx
  - FULL HOLD + ADD: 202 stocks
✓ PDF report created: c:\workspace\my_script_project\scanner_results\scanner_report_sp500_20260106_1456.pdf
  - 202 FULL HOLD + ADD (detailed)
  ✓ Excel: sp500_analysis_20260106_1456.xlsx
  ✓ PDF: scanner_report_sp500_20260106_1456.pdf

🎯 NASDAQ 100: 42 FULL HOLD + ADD signals found
✓ Excel file created: c:\workspace\my_script_project\scanner_results\nasdaq100_analysis_20260106_1456.xlsx
  - FULL HOLD + ADD: 42 stocks
✓ PDF report created: c:\workspace\my_script_project\scanner_results\scanner_report_nasdaq100_20260106_1456.pdf
  - 42 FULL HOLD + ADD (detailed)
  ✓ Excel: nasdaq100_analysis_20260106_1456.xlsx
  ✓ PDF: scanner_report_nasdaq100_20260106_1456.pdf

🎯 Portfolio: 13 stocks scanned (all included)
  - 5 with FULL HOLD + ADD signal
✓ Excel file created: c:\workspace\my_script_proj

---
## Summary & Results
Final scan statistics and file locations

In [6]:
# ============================================================================
# CELL 12: Summary Statistics - Shows total signals found and timing
# ============================================================================

print("\n" + "="*80)
print("SCAN COMPLETE - SUMMARY")
print("="*80)

total_time = sp500_elapsed + nasdaq100_elapsed + portfolio_elapsed
print(f"⏱️  Total time: {total_time:.1f}s\n")

print("📊 Results by Category:")
if not sp500_results.empty:
    sp500_buy = filter_buy_signals(sp500_results, 'FULL HOLD + ADD')
    print(f"  S&P 500:     {len(sp500_buy):3d} FULL HOLD + ADD signals ({sp500_elapsed:.1f}s)")
if not nasdaq100_results.empty:
    nasdaq100_buy = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD')
    print(f"  NASDAQ 100:  {len(nasdaq100_buy):3d} FULL HOLD + ADD signals ({nasdaq100_elapsed:.1f}s)")
if not portfolio_results.empty:
    portfolio_buy = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD')
    print(f"  Portfolio:    {len(portfolio_buy):3d} FULL HOLD + ADD signals ({portfolio_elapsed:.1f}s)")

print("\n✅ All reports generated in scanner_results/")


SCAN COMPLETE - SUMMARY
⏱️  Total time: 708.2s

📊 Results by Category:
  S&P 500:     202 FULL HOLD + ADD signals (678.7s)
  NASDAQ 100:   42 FULL HOLD + ADD signals (24.5s)
  Portfolio:      5 FULL HOLD + ADD signals (5.0s)

✅ All reports generated in scanner_results/
